In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

from poliastro.bodies import Sun, Earth, Jupiter, Mars
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import StaticOrbitPlotter
from poliastro.util import norm, time_range
from poliastro import iod
from poliastro.plotting import OrbitPlotter3D
from poliastro.threebody.flybys import compute_flyby

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
## Main dates (1 набор дат)
date_launch = Time("2029-07-06", scale="utc").tdb
date_flyby_Mars = Time("2030-12-30", scale="utc").tdb
date_arrival = Time("2035-06-01", scale="utc").tdb

In [3]:
earth_1 = Ephem.from_body(Earth, time_range(date_launch, end=date_launch))
mar = Ephem.from_body(Mars, time_range(date_launch, end=date_flyby_Mars))
jup = Ephem.from_body(Jupiter, time_range(date_launch, end=date_arrival))

In [4]:
ss_earth_1 = Orbit.from_ephem(Sun, earth_1, date_launch)
ss_mar = Orbit.from_ephem(Sun, mar, date_flyby_Mars)
ss_jup = Orbit.from_ephem(Sun, jup, date_arrival)

In [5]:
man_lambert_1 = Maneuver.lambert(ss_earth_1, ss_mar)
ss_trans_1, ss_target_1 = ss_earth_1.apply_maneuver(man_lambert_1, intermediate=True)

man_lambert_2 = Maneuver.lambert(ss_mar, ss_jup)
ss_trans_2, ss_target_2 = ss_mar.apply_maneuver(man_lambert_2, intermediate=True)

In [6]:
#1
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth")
plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")


plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=241 * u.deg), color="blue", label="Transfer orbit E-M"
)

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=202 * u.deg), color="purple", label="Transfer orbit M-E"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [7]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth")
plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")

plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=241 * u.deg), color="blue", label="Transfer orbit E-M"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [8]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=202 * u.deg), color="purple", label="Transfer orbit M-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [9]:
r_earth, v_earth = Ephem.from_body(Earth, date_launch).rv(date_launch)
r_mar, v_mar = Ephem.from_body(Mars, date_flyby_Mars).rv(date_flyby_Mars)
r_jup, v_jup = Ephem.from_body(Jupiter, date_arrival).rv(date_arrival)

In [10]:
# Фазовые координаты Земли_1

print (f"{r_earth [0]}")
print (f"{r_earth [1]}")
print (f"{r_earth [2]}"'\n')

print (v_earth.to(u.km / u.s))

36480176.949702546 km
-135575476.11002797 km
-58763701.68457585 km

[28.44547638  6.42577066  2.78595158] km / s


In [11]:
# Фазовые координаты Марса

print (f"{r_mar [0]}")
print (f"{r_mar [1]}")
print (f"{r_mar [2]}"'\n')

print (v_mar.to(u.km / u.s))

-241371146.80995303 km
53080629.257726066 km
30857580.29783413 km

[ -5.02599382 -19.52431941  -8.8199922 ] km / s


In [27]:
# Фазовые координаты Юпитера

print (f"{r_jup [0]}")
print (f"{r_jup [1]}")
print (f"{r_jup [2]}"'\n')

print (v_jup.to(u.km / u.s))

638327719.1660622 km
351852877.53404033 km
135276695.11736917 km

[-6.79796303 10.84038031  4.81185506] km / s


In [28]:
nh_tof_1 = date_flyby_Mars - date_launch # время полета Земля1 - Венера

(v0_Ea1, vk_mar), = iod.lambert(Sun.k, r_earth, r_mar, nh_tof_1)
nh_tof_1

<TimeDelta object: scale='tdb' format='jd' value=541.999999998871>

In [29]:
# Гиперболический избыток скорости при отлете от Земли

V_inf = (norm(v0_Ea1 - v_earth)).to(u.km / u.s)
V_inf

<Quantity 5.05034143 km / s>

In [30]:
# Гиперболический избыток скорости при отлете от Земли

V_inf_vec = (v0_Ea1 - v_earth).to(u.km / u.s)
V_inf_vec

<Quantity [ 4.60689416,  2.04176263, -0.33716478] km / s>

In [31]:
# Гиперболический избыток скорости при подлете к Марсу

V_inf_end = (norm(vk_mar - v_mar)).to(u.km / u.s)
V_inf_end

<Quantity 7.15244578 km / s>

In [32]:
# Гиперболический избыток скорости при подлете к Марсу

V_inf_end_vec = (vk_mar - v_mar).to(u.km / u.s)
V_inf_end_vec

<Quantity [ 7.10862689, -0.77858558,  0.13678021] km / s>

In [33]:
nh_tof_2 = date_arrival - date_flyby_Mars # время полета Марс - Земля2

(v0_mar, vk_Jup), = iod.lambert(Sun.k, r_mar, r_jup, nh_tof_2)
nh_tof_2

<TimeDelta object: scale='tdb' format='jd' value=1614.0000000124867>

In [34]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_end_vec_0 = (norm(v0_mar - v_mar)).to(u.km / u.s)
V_inf_end_vec_0

<Quantity 7.17159702 km / s>

In [35]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_end_vec_0_vec = (v0_mar - v_mar).to(u.km / u.s)
V_inf_end_vec_0_vec

<Quantity [ 1.93785639, -6.42867603, -2.51965097] km / s>

In [56]:
# Разница подлетного и отлетного гиперболического избытка скорости

V_gip_diff = V_inf_end - V_inf_end_vec_0
V_gip_diff

<Quantity -0.01915125 km / s>

In [36]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_end_2 = (norm(vk_Jup - v_jup)).to(u.km / u.s)
V_inf_end_2

<Quantity 5.74676724 km / s>

In [38]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_end_2_vec = (vk_Jup - v_jup).to(u.km / u.s)
V_inf_end_2_vec

<Quantity [-1.253508  , -5.20802065, -2.08100274] km / s>

In [39]:
time_adv = nh_tof_1 + nh_tof_2
time_adv

<TimeDelta object: scale='tdb' format='jd' value=2156.0000000113573>

In [40]:
scalar_EM = r_earth[0]*r_mar[0] + r_earth[1]*r_mar[1] +r_earth[2]*r_mar[2] 
scalar_MJ = r_mar[0]*r_jup[0] + r_mar[1]*r_jup[1] +r_mar[2]*r_jup[2]

In [43]:
fi_EM = (scalar_EM/norm(r_earth)/norm(r_mar))
fi_MJ = (scalar_MJ/norm(r_mar)/norm(r_jup))

In [44]:
np.degrees(np.arccos(fi_EM))

<Quantity 118.03251568 deg>

In [45]:
np.degrees(np.arccos(fi_MJ))

<Quantity 135.29390134 deg>

In [48]:
## Необходимый импульс на опорной орбите 
from math import*

mu = 398600.4415 ## Гравитационный параметр Земли 
r = 6371.
R = r + 200.
V_gip = 5.0503414 ## Гиперболический избыток скорости при отлете от Земли

dV = sqrt (2 * mu / R + V_gip**2 ) - sqrt(mu/r)
dV * u.km / u.s

<Quantity 4.20742834 km / s>

In [49]:
## Необходимый импульс для торможения на эллиптическую орбиту Юпитера 
## Не уверен что это нужно

muJ = 126686534
r_p = 670900
r_a = 4000000
V_inf_jup = 5.7467672

print (sqrt ((2 * mu / R + V_inf_jup**2 )))
print (sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p))))

dV_el = sqrt ((2 * mu / R + V_inf_jup**2 )) - sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p)))
dV_el * u.km / u.s

12.423623599571423
17.983782441612995


<Quantity -5.56015884 km / s>

In [54]:
## параметры участка Землия-Марс 

from math import*

to_rad = pi/180

ist_an_1 = 356.1164273 * to_rad
ist_an_2 = 238.4012110 * to_rad

exs_EM = (norm(r_mar)/norm(r_earth) - 1) / (cos(ist_an_1) - (norm(r_mar)/norm(r_earth) * cos(ist_an_2)))
p_EM = norm(r_earth)*(1+cos(ist_an_1)*exs_EM)
a_EM = p_EM/(1 - exs_EM*exs_EM)

print (f"Эксцентриситет {exs_EM}")
print (f"Фокальный параметр {p_EM}")
print (f"Большая полуось {a_EM}")

Эксцентриситет 0.3430454000209822
Фокальный параметр 204290938.0752738 km
Большая полуось 231538412.35605156 km


In [55]:
## параметры участка Марс-Юпитер 

ist_an_1_MJ = 337.3677377 * to_rad
ist_an_2_MJ = 201.7045516 * to_rad

exs_MJ = (norm(r_jup)/norm(r_mar) - 1) / (cos(ist_an_1_MJ) - (norm(r_jup)/norm(r_mar) * cos(ist_an_2_MJ)))
p_MJ = norm(r_mar)*(1+cos(ist_an_1_MJ)*exs_MJ)
a_MJ = p_MJ/(1 - exs_MJ*exs_MJ)

print (f"Эксцентриситет {exs_MJ}")
print (f"Фокальный параметр {p_MJ}")
print (f"Большая полуось {a_MJ}")

Эксцентриситет 0.5358615281605189
Фокальный параметр 372240950.6156711 km
Большая полуось 522185151.9258666 km
